In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from windmark.core.structs import LevelSet

In [8]:
levelset = LevelSet("event_type", ["hello", "world", "oh_yeah"])

In [12]:
levelset.mapping

<enum 'LevelEnum'>